In [387]:
import requests
from bs4 import BeautifulSoup
import re
import urllib
import pandas as pd
import numpy as np

All data is imported from http://isophonics.net/content/reference-annotations-beatles. RDF file can be played with mp3 using https://www.sonicvisualiser.org/.

In [38]:
req = requests.get('http://isophonics.net/content/reference-annotations-beatles')
soup = BeautifulSoup(req.text)

In [276]:
#Parse url in order to find links to download csv

pattern = '[0-9\.\'\(\)A-Za-z- ]+'
songs = []
structural_segmentation = []
key_changes = []
chords = []
beats = []
for i in range(len(soup.find_all('ol'))):
    for k in range(len(soup.find_all('ol')[i].find_all('li'))):
        chunk = soup.find_all('ol')[i].find_all('li')[k]
        title = chunk.text
        song = re.findall(pattern, title)[0]
        if (song == 'Structural segmentation'):
            structural_segmentation.append(chunk.find('a').get('href'))
        elif (song == 'Key changes'):
            key_changes.append(chunk.find('a').get('href'))
        elif (song == 'Chords'):
            chords.append(chunk.find('a').get('href'))
        elif (song == 'Beats'):
            beats.append(chunk.find('a').get('href'))
        elif (song != 'All of the above'):
                songs.append(song)
                if song == 'Revolution 9':
                    key_changes.append('none')
                    beats.append('none')
                    
                              

In [421]:
song_info = pd.DataFrame({'songs':songs, 'structural_segmentation': structural_segmentation, 'key_changes': key_changes, 'chords': chords, 'beats':beats})
def get_info(song, data):
    """
    Inputs song name and type of data needed, returns link to input into following functions.
    """
    return song_info.loc[song_info["songs"] == song].get(data).item()


In [464]:
def parse_structural_segmentation(inp):
    """
    Inputs url to txt file for structural segmentation and outputs dataframe. 
    """
    inp = inp.replace(' ', '%20')
    url = 'http://isophonics.net' + inp
    file = urllib.request.urlopen(url)
    data = pd.read_csv(file, sep="\t", header=None, names=list(range(5)))
    return data
    data = data.drop(2, axis=1)
    data.columns = ['begin', 'end', 'structure', 'note']
    return data

In [360]:
def parse_key_changes(inp):
    """
    Inputs url to txt file for key changes and outputs dataframe. 
    """
    inp = inp.replace(' ', '%20')
    url = 'http://isophonics.net' + inp
    file = urllib.request.urlopen(url)
    data = pd.read_csv(file, sep="\t", header=None, names=list(range(2)))
    data.columns = ['begin', 'end', 'key or silence', 'key']
    return data

In [453]:
def parse_beats(inp):
    """
    Inputs url to txt file for beats and outputs dataframe. 
    """
    inp = inp.replace(' ', '%20')
    url = 'http://isophonics.net' + inp
    file = urllib.request.urlopen(url)
    data = pd.read_csv(file, sep="\t", header=None, names=list(range(2)))
    data.columns = ['time', 'beats']
    return data
    if data["beats"][0] != "New Point":
        if np.isnan(int(data["beats"][0])):
            spl = data["time"].apply(lambda x: x.split())
            data["time"] = spl.apply(lambda x: x[0])
            data["beats"] = spl.apply(lambda x: x[1])
    return data

In [419]:
def parse_chords(inp):
    """
    Inputs url to txt file for chords and outputs dataframe. 
    """
    inp = inp.replace(' ', '%20')
    url = 'http://isophonics.net' + inp
    file = urllib.request.urlopen(url)
    data = pd.read_csv(file, sep=" ", header=None, names=list(range(3)))
    data.columns = ['begin', 'end', 'chord']
    return data

In [465]:
path = '../data/metadata/'
boole = False
for song in song_info["songs"]:
    if song == 'Mr. Moonlight':
        boole = True
    if boole == True:
        ss_link = get_info(song, "structural_segmentation")
        ss_df = parse_structural_segmentation(ss_link)
        ss_df.to_csv(path+'structural_segmentation/' + song + '_ss.csv', header=True, index=False)
        if get_info(song, "key_changes") != 'none':
            kc_link = get_info(song, "key_changes")
            kc_df = parse_key_changes(kc_link)
            kc_df.to_csv(path+'key_changes/' + song + '_kc.csv', header=True, index=False)
        if get_info(song, "beats") != 'none':
            beats_link = get_info(song, "beats")
            beats_df = parse_beats(beats_link)
            beats_df.to_csv(path+'beats/' + song + '_beats.csv', header=True, index=False)
        chords_link = get_info(song, "chords")
        chords_df = parse_chords(chords_link)
        chords_df.to_csv(path+'chords/' + song + '_chords.csv', header=True, index=False)

        

http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/06_-_Mr._Moonlight.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/07_-_Kansas_City-_Hey,_Hey,_Hey,_Hey.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/08_-_Eight_Days_a_Week.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/09_-_Words_of_Love.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/10_-_Honey_Don't.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/11_-_Every_Little_Thing.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/12_-_I_Don't_Want_to_Spoil_the_Party.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/13_-_What_You're_Doing.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/04_-_Beatles_for_Sale/14_-_Everybody's_Tryin

http://isophonics.net/files/annotations/keylab/The%20Beatles/10CD1_-_The_Beatles/CD1_-_01_-_Back_in_the_USSR.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/10CD1_-_The_Beatles/CD1_-_02_-_Dear_Prudence.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/10CD1_-_The_Beatles/CD1_-_03_-_Glass_Onion.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/10CD1_-_The_Beatles/CD1_-_04_-_Ob-La-Di,_Ob-La-Da.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/10CD1_-_The_Beatles/CD1_-_05_-_Wild_Honey_Pie.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/10CD1_-_The_Beatles/CD1_-_06_-_The_Continuing_Story_of_Bungalow_Bill.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/10CD1_-_The_Beatles/CD1_-_07_-_While_My_Guitar_Gently_Weeps.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/10CD1_-_The_Beatles/CD1_-_08_-_Happiness_is_a_Warm_Gun.lab
http://isophonics.net/files/annotations/keylab/The%20Beatles/10C